<a href="https://colab.research.google.com/github/minoooo119/2023webprogramming_final_project/blob/main/LLM_2024_spring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
####################### 인공지능 프로그래밍(AI Project), SWE3032-41########################
#################### Week7 - LLM ######################
###############################################################

# Due Date - 2024.04.21 11:59 pm
# TA - Jeong Hongjun
# E-mail - zun8861@g.skku.edu


## HuggingFace
https://huggingface.co/
- Hugging Face is a company that specializes in Natural Language Processing (NLP) and provides a comprehensive set of tools and libraries for working with state-of-the-art models.
- One of their most popular contributions is the transformers library, which includes pre-trained models and utilities for training and fine-tuning models like BERT. The library simplifies the process of working with transformer-based models, making them accessible to a broader audience.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Start by creating a random folder in google drive.
# google drive에 임의의 폴더를 만들어서 진행합니다.
%mkdir ./drive/MyDrive/LLM
%cd ./drive/MyDrive/LLM

mkdir: cannot create directory ‘./drive/MyDrive/LLM’: File exists
/content/drive/MyDrive/LLM


In [4]:
import torch
import torchvision
print(f'torch version: {torch.__version__}')
print(f'torchvision version: {torchvision.__version__}')


torch version: 2.2.1+cu121
torchvision version: 0.17.1+cu121


### Data Preparation : (Korpora)
Korpora is an open-source Python library that makes it easy to load datasets for Korean natural language processing.  
Korpora는 한글 자연어처리를 위한 데이터셋을 손쉽게 로드해주도록 도와주는 오픈소스 파이썬 라이브러리이다.
- https://github.com/ko-nlp/Korpora

In [1]:
# install 1 : from source
# !git clone https://github.com/ko-nlp/Korpora
# !python setup.py install

# install 2 : using pip
!pip install Korpora==0.2.0rc1


!pip install transformers
!pip install datasets
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.6 MB/s eta 0:00:00


In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Korean natural language processing dataset
# 한글 자연어 처리 데이터셋
from Korpora import Korpora

# Set to ignore warnings about tokenizer
# 토크나이저 관련 경고 무시하기 위하여 설정
os.environ["TOKENIZERS_PARALLELISM"] = 'true'

# specify device
# device 지정
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'device: {device}')

corpus = Korpora.load("nsmc")

device: cuda:0

    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:00, 133MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 58.9MB/s]


Proceed to Korpora's Naver movie comment dataset.  
Korpora의 네이버 영화 댓글 데이터셋으로 진행한다


- https://github.com/e9t/nsmc

In [3]:
import pandas as pd

train = pd.read_csv('~/Korpora/nsmc/ratings_train.txt', sep='\t')
test = pd.read_csv('~/Korpora/nsmc/ratings_test.txt', sep='\t')
print('train',train.shape)
print('test',test.shape)

train (150000, 3)
test (50000, 3)


In [4]:
train.head(10)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


### Data preprocessing and tokenizing
### 데이터 전처리 및 토큰화

Calculate the length of the sentence and put it in the [length] column.  
문장의 길이를 계산하여 [length] column에 담는다.

In [5]:
train['length'] = train['document'].apply(lambda x: len(str(x)))
test['length'] = test['document'].apply(lambda x: len(str(x)))


In [6]:
train.head(10)

,id,document,label,length
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,19
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,33
2,10265843,너무재밓었다그래서보는것을추천한다,0,17
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,29
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,61
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0,45
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0,21
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0,86
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1,22
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1,45


Only gets sentences with a length of 5 or more words.
문장의 길이가 단어 기준으로 5이상인 문장만 가져온다.

- loc: Short for location, indexing by a label or boolean array of rows or columns in a dataframe.
    - How to access human-readable data by writing column names or specific conditional expressions.


- loc : location의 약자로 데이터프레임의 행 또는 칼럼의 label이나 boolean array로 인덱싱하는 방법.
    - 칼럼명을 직접 적거나 특정 조건식을 써주어 사람이 읽을 수 있게 데이터에 접근하는 방법
    

In [7]:
train = train.loc[train['length'] > 5]

# Sample 1000 sentences because the overall dataset size is large.
# 전체 데이터셋 크기가 커서 1000개의 문장을 샘플링 합니다.
train = train.sample(1000)
train.head()

,id,document,label,length
77064,8469718,나혼자산다 애청잔데 요번설특집은 뭔지..이미방영한거를 특집으로 보여주시는데..이게무...,0,132
46919,9830369,와 너무하네.. 진짜 슬프다,0,15
118714,7132049,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,0,14
7118,10032442,내 인생 최고의 영화. 100점도 아깝지 않다.,1,26
126505,10182157,중학생인데 중학생이 봐도 볼께업따..ㅜ겁나 뻔한스토리랑 스토리 다썪어서 아주그냥 쌈...,0,81


In [8]:
test = test.loc[test['length'] > 5]
test = test.sample(1000)
test.head()

,id,document,label,length
16024,7016410,잼난다~독특한 이런느낌 영화 좋다~지루하지않아~!,1,27
38997,9857048,역사상 가장 위대한 다큐,1,13
47467,2057843,이영화의 제목 : 야동보는 엄마,0,17
42636,3642994,오랫만에 잔잔한영화한편봤네요..,1,17
44996,7951287,진심클라라아니다.. 말도행동도다른사람..섭외하기전 그사람에대해 파악좀했으면 ..있는...,0,58


In [9]:
MODEL_NAME = 'kykim/bert-kor-base'

In [10]:
import torch
from transformers import BertTokenizerFast
from torch.utils.data import Dataset, DataLoader


class NaverReviewDataset(Dataset):

    def __init__(self, dataframe, tokenizer_pretrained):
        # pass a dataframe with sentence and label columns
        # sentence, label 컬럼으로 구성된 데이터프레임 전달
        self.data = dataframe

        # Create a Huggingface tokeniezr
        # Huggingface 토크나이저 생성 -> 토큰 단위로 끊어준다
        self.tokenizer = BertTokenizerFast.from_pretrained(tokenizer_pretrained)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # iloc: how to select by line number
        # iloc : 행번호로 선택하는 방법
        sentence = self.data.iloc[idx]['document']
        label = self.data.iloc[idx]['label']

        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                # sentence(1개 문장)
            return_tensors='pt',     # Return PyTorch torch.Tensor objects(pytorch 텐서로 반환)
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )

        input_ids = tokens['input_ids'].squeeze(0)       # 2D -> 1D : 1,512 -> 512
        attention_mask = tokens['attention_mask'].squeeze(0) # 2D -> 1D
        token_type_ids = torch.zeros_like(attention_mask)

        # make it return three elements: input_ids, attention_mask, and token_type_ids.
        # input_ids, attention_mask, token_type_ids 이렇게 3가지 요소를 반환하도록 합니다.
        # input_ids: token
        # attention_mask: 1 if the actual word exists, 0 if padding (padding may not be zero)
        # token_type_ids: IDs that separate sentences. All 0 for single sentences.
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'token_type_ids': token_type_ids,
        }, torch.tensor(label)

In [11]:
# Specify a tokenizer
# 토크나이저 지정
tokenizer_pretrained = MODEL_NAME

# Create train, test datasets
# train, test 데이터셋 생성
train_data = NaverReviewDataset(train, tokenizer_pretrained)
test_data = NaverReviewDataset(test, tokenizer_pretrained)

# Specify the Dataset you created earlier with DataLoader, setting the batch configuration, shuffle, num_workers, etc.
# DataLoader로 이전에 생성한 Dataset를 지정하여, batch 구성, shuffle, num_workers 등을 설정합니다.
train_loader = DataLoader(train_data, batch_size=8, shuffle=True, num_workers=2)
test_loader = DataLoader(test_data, batch_size=8, shuffle=False, num_workers=2)

# Train_loader and test_loader are iterators.
# An iterator is an object that uses repetition to perform some processing.
# train_loader과 test_loader는 iterator이다.
# iterator란 반복을 이용해 어떤 처리를 수행하는 객체를 말한다.
print('train_loader',train_loader)
print('test_loader',test_loader)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/344k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

train_loader <torch.utils.data.dataloader.DataLoader object at 0x7cfb652599f0>
test_loader <torch.utils.data.dataloader.DataLoader object at 0x7cfb50330940>


Pull only 1 sample from the created train_loader and check the value.  
생성된 train_loader로부터 1개의 sample만 꺼내서 값을 확인



In [12]:
inputs, labels = next(iter(train_loader))

# Set the device for the dataset
# 데이터셋의 device 설정
inputs = {k: v.to(device) for k, v in inputs.items()}


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


- input_ids : token
- attention_mask : 1 if the actual word exists, 0 if padding (padding may not be zero)
- token_type_ids : IDs that separate sentences. All 0 for single sentences.

In [13]:
# output the key value of the generated inputs
# 생성된 inputs의 key값 출력
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'token_type_ids'])

In [14]:
# check shape of each input's key
# key 별 shape 확인
inputs['input_ids'].shape, inputs['attention_mask'].shape, inputs['token_type_ids'].shape

(torch.Size([8, 512]), torch.Size([8, 512]), torch.Size([8, 512]))

In [15]:
from transformers import BertConfig

config = BertConfig.from_pretrained(MODEL_NAME)
config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "embedding_size": 768,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.38.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 42000
}

In [16]:
# print label
# label 출력
print(labels)

tensor([1, 1, 0, 0, 1, 0, 1, 0])


### Pretrained Model

In [17]:
from transformers import BertModel

# Create a model
# 모델 생성
model = BertModel.from_pretrained(MODEL_NAME).to(device)
model

pytorch_model.bin:   0%|          | 0.00/476M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(42000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

If you put the result in output and print the key, you'll see two outputs: 'last_hidden_state' and 'pooler_output'.
결과를 output 에 담은 후 key를 출력해 보면 'last_hidden_state' 와 'pooler_output'이렇게 2개가 출력됩니다.  
- last_hidden_state contains the hidden representation for each token in each sequence in the batch, so its size is (batch_size, seq_len, hidden_size).
- last_hidden_state는 배치의 각 시퀀스에서 각 토큰에 대한 숨겨진 표현을 포함합니다. 따라서 크기는 (batch_size, seq_len, hidden_size)입니다.  

- pooler_output contains a "representation" of each sequence in the batch and is of size (batch_size, hidden_size). Basically, what we're doing is taking the hidden representation of each sequence's [CLS] token from the batch (a vector of size hidden_size) and running it through the BertPooler. This consists of a linear layer and a Tanh activation function.  
- pooler_output은 배치의 각 시퀀스의 “표현”을 포함하며 크기(batch_size, hidden_size)입니다. 기본적으로 하는 일은 배치(hidden_size 크기의 벡터)에서 각 시퀀스의 [CLS] 토큰의 숨겨진 표현을 가져온 다음 BertPooler를 통해 실행하는 것입니다. 이것은 선형 레이어와 Tanh 활성화 함수로 구성됩니다.

In [18]:
output = model(**inputs)
output.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [19]:
print(output['last_hidden_state'].shape) #embadding size 768임
print(output['pooler_output'].shape)


torch.Size([8, 512, 768])
torch.Size([8, 768])


In [20]:
print(output['last_hidden_state'][:,0,:])


tensor([[-0.0865,  0.0254, -0.0527,  ...,  0.1327, -0.3664,  0.0355],
        [-0.4387,  0.2375, -0.2768,  ..., -1.6341, -0.3997, -0.9031],
        [ 0.2561,  0.0567,  0.6367,  ..., -0.7163, -1.5010,  0.1826],
        ...,
        [ 0.0807, -0.0602, -0.4715,  ..., -0.7455, -0.4280, -0.8243],
        [ 0.0080,  0.0381,  0.3992,  ...,  0.2035, -0.6703,  0.2410],
        [ 0.4182,  0.0696,  0.4657,  ..., -0.7395, -0.2205,  0.2176]],
       device='cuda:0', grad_fn=<SliceBackward0>)


In [21]:
print(output['pooler_output'])

tensor([[-0.7381, -0.0517, -0.8991,  ..., -0.7071,  0.0928,  0.7868],
        [ 0.8035,  0.4227, -0.9984,  ..., -0.6130,  0.4127,  0.0749],
        [ 0.9552,  0.0043, -0.9991,  ..., -0.6952,  0.6223, -0.3142],
        ...,
        [ 0.6266,  0.4489, -0.9732,  ..., -0.9689,  0.5117,  0.1111],
        [-0.8143, -0.0691, -0.8596,  ..., -0.9536,  0.2131,  0.5601],
        [-0.6751,  0.3513, -0.8778,  ..., -0.1956,  0.0121,  0.8386]],
       device='cuda:0', grad_fn=<TanhBackward0>)


In [22]:
fc = nn.Linear(768, 2)
fc.to(device)
fc_output = fc(output['last_hidden_state'][:, 0, :])
print(fc_output.shape)
print(fc_output)
print(fc_output.argmax(dim=1))

torch.Size([8, 2])
tensor([[ 0.0769, -0.4387],
        [-0.9638,  0.3084],
        [-0.4123,  0.4230],
        [ 0.2159,  0.0503],
        [-0.5823,  0.6377],
        [-0.4238,  0.1383],
        [-0.3638, -0.0559],
        [-0.0933, -0.5504]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([0, 1, 1, 0, 1, 1, 1, 0], device='cuda:0')


### Let's fine-tuning

In [23]:
class FineTuningBertModel(nn.Module):
    def __init__(self, bert_pretrained, dropout_rate=0.5):
        super().__init__()

        # specify pretrained model
        # 사전학습 모델 지정
        self.bert = BertModel.from_pretrained(bert_pretrained)

        # specify dropout
        # dropout 설정
        self.dropout = nn.Dropout(p=dropout_rate)

        # define output layer
        # 최종 출력층 정의
        self.fc = nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        last_hidden_state = output['last_hidden_state']
        # get [CLS] token
        # last_hidden_state[:, 0, :]는 [CLS] 토큰을 가져옴
        x = self.dropout(last_hidden_state[:, 0, :])
        x = self.fc(x)
        return x

In [24]:
BERT_model = FineTuningBertModel(MODEL_NAME)
BERT_model.to(device)

FineTuningBertModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

Loss & Optimizer

In [25]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(BERT_model.parameters(), lr=1e-5)

In [26]:
from tqdm import tqdm  # Progress Bar 학습이 얼마나 진행되는지 알 수 잇음 중요한거 아님

def model_train(model, data_loader, loss_fn, optimizer, device):
    # Set the model to training mode. Gradient will be updated when in training mode. Be sure to change mode with train().
    # 모델을 훈련모드로 설정합니다. training mode 일 때 Gradient 가 업데이트 됩니다. 반드시 train()으로 모드 변경을 해야 합니다.
    model.train()

    # This is a temporary variable for calculating loss and accuracy. Initialize it to 0.
    # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
    running_loss = 0
    corr = 0
    counts = 0

    # Wrap it in tqdm to monitor the training status while outputting a pretty progress bar.
    # 예쁘게 Progress Bar를 출력하면서 훈련 상태를 모니터링 하기 위하여 tqdm으로 래핑합니다.
    prograss_bar = tqdm(data_loader, unit='batch', total=len(data_loader), mininterval=1)

    # Start learning mini-batch.
    # mini-batch 학습을 시작합니다.
    for idx, (inputs, labels) in enumerate(prograss_bar):
        # put inputs, label data on device (cuda:0 or cpu)
        # inputs, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
        inputs = {k:v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)

        # Initialize the stacked Gradient.
        # 누적 Gradient를 초기화 합니다.
        optimizer.zero_grad()

        # Proceed with Forward Propagation to get the result.
        # Forward Propagation을 진행하여 결과를 얻습니다.
        output = model(**inputs) #2차원 아웃풋일 거야

        # Calculate the loss by substituting the values of output, label into the loss function.
        # 손실함수에 output, label 값을 대입하여 손실을 계산합니다.
        loss = loss_fn(output, labels)

        # Proceed with back propagation to compute the derivative.
        # 오차역전파(Back Propagation)을 진행하여 미분 값을 계산합니다.
        loss.backward()

        # update calculated gradient
        # 계산된 Gradient를 업데이트 합니다.
        optimizer.step()

        # output.max(dim=1) returns max probability and max index
        # Ignore max probability, and store max index in index to compare against the label value to derive the accuracy.
        # output의 max(dim=1)은 max probability와 max index를 반환합니다.
        # max probability는 무시하고, max index는 index에 저장하여 label 값과 대조하여 정확도를 도출합니다.
        _, index = output.max(dim=1)

        # index.eq(lbl).sum() computes the sum of the labels that match exactly. item() extracts a value from a tensor.
        # accumulate the sum in the corr variable.
        # index.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
        # 합계는 corr 변수에 누적합니다.
        corr += index.eq(labels).sum().item()
        counts += len(labels)

        # loss value is the average loss for one batch. labels.size(0) is the batch size.
        # multiply loss by labels.size(0) to calculate the total loss for one batch.
        # accumulate them and divide by the total number of datasets at the end of the epoch to get the mean loss.
        # loss 값은 1개 배치의 평균 손실(loss) 입니다. labels.size(0)은 배치사이즈(batch size) 입니다.
        # loss 와 labels.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
        # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
        running_loss += loss.item() * labels.size(0)

        # Update learning progress in the progress bar
        # 프로그레스바에 학습 상황 업데이트
        prograss_bar.set_description(f"training loss: {running_loss/(idx+1):.5f}, training accuracy: {corr / counts:.5f}")

    # The accumulated number of correct answers divided by the total number yields the accuracy.
    # 누적된 정답수를 전체 개수로 나누어 주면 정확도가 산출됩니다.
    acc = corr / len(data_loader.dataset)

    # Returns the mean loss and accuracy.
    # 평균 손실(loss)과 정확도를 반환합니다.
    # train_loss, train_acc
    return running_loss / len(data_loader.dataset), acc

In [27]:
def model_evaluate(model, data_loader, loss_fn, device):
    # model.eval() sets the model to eval mode.
    # This is a necessary step in the evaluation process to change the role of a layer such as dropout.
    # model.eval()은 모델을 평가모드로 설정을 바꾸어 줍니다.
    # dropout과 같은 layer의 역할 변경을 위하여 evaluation 진행시 꼭 필요한 절차 입니다.
    model.eval()

    # This is required to prevent the gradient from being updated.
    # Gradient가 업데이트 되는 것을 방지 하기 위하여 반드시 필요합니다.
    with torch.no_grad():
        # This is a temporary variable for calculating loss and accuracy. Initialize it to 0.
        # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
        corr = 0
        running_loss = 0

        # Proceed evaluation for each batch.
        # 배치별 evaluation을 진행합니다.
        for inputs, labels in data_loader:
            # put inputs, label data on device (cuda:0 or cpu)
            # inputs, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
            inputs = {k:v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)

            # Proceed with Forward Propagation to get the result.
            # 모델에 Forward Propagation을 하여 결과를 도출합니다.
            output = model(**inputs) # (8,2)


            # output.max(dim=1) returns max probability and max index
            # Ignore max probability, and store max index in index to compare against the label value to derive the accuracy.
            # output의 max(dim=1)은 max probability와 max index를 반환합니다.
            # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
            _, index = output.max(dim=1)

            # index.eq(lbl).sum() computes the sum of the labels that match exactly. item() extracts a value from a tensor.
            # accumulate the sum in the corr variable.
            # index.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
            # 합계는 corr 변수에 누적합니다.
            corr += torch.sum(index.eq(labels)).item()


            # loss value is the average loss for one batch. labels.size(0) is the batch size.
            # multiply loss by labels.size(0) to calculate the total loss for one batch.
            # accumulate them and divide by the total number of datasets at the end of the epoch to get the mean loss.
            # loss 값은 1개 배치의 평균 손실(loss) 입니다. img.size(0)은 배치사이즈(batch size) 입니다.
            # loss 와 img.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
            # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
            running_loss += loss_fn(output, labels).item() * labels.size(0)

        # Calculate the validation accuracy.
        # Divide the accumulated number of correct answers by the number in the entire dataset to get the final accuracy.
        # validation 정확도를 계산합니다.
        # 누적한 정답숫자를 전체 데이터셋의 숫자로 나누어 최종 accuracy를 산출합니다.
        acc = corr / len(data_loader.dataset)

        # return outputs
        # 결과를 반환합니다.
        # val_loss, val_acc
        return running_loss / len(data_loader.dataset), acc

Training Loop

In [28]:
# specify epochs
# 최대 Epoch을 지정합니다.
num_epochs = 5

# Define the name of the model to save as a checkpoint.
# checkpoint로 저장할 모델의 이름을 정의 합니다.
model_name = 'bert-kor-base'

min_loss = np.inf

# Perform training and validation.
# Epoch 별 훈련 및 검증을 수행합니다.
for epoch in range(num_epochs):
    # Model Training
    # 훈련 손실과 정확도를 반환 받습니다.
    train_loss, train_acc = model_train(BERT_model, train_loader, loss_fn, optimizer, device)

    # 검증 손실과 검증 정확도를 반환 받습니다.
    val_loss, val_acc = model_evaluate(BERT_model, test_loader, loss_fn, device)

    # If val_loss has improved, update min_loss and save the model's weights.
    # val_loss 가 개선되었다면 min_loss를 갱신하고 model의 가중치(weights)를 저장합니다.
    if val_loss < min_loss:
        print(f'[INFO] val_loss has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!')
        min_loss = val_loss
        torch.save(BERT_model.state_dict(), f'{model_name}.pth')
    # print results for each epoch
    # Epoch 별 결과를 출력합니다.
    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, acc: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}')

  0%|          | 0/125 [00:00<?, ?batch/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
training loss: 5.26803, training accuracy: 0.64600: 100%|██████████| 125/125 [01:32<00:00,  1.37batch/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
training loss: 5.26803, training accuracy: 0.64600: 100%|██████████| 125/125 [01:32<00:00,  1.34batch/s]


[INFO] val_loss has been improved from inf to 0.47659. Saving Model!
epoch 01, loss: 0.65850, acc: 0.64600, val_loss: 0.47659, val_accuracy: 0.80800


training loss: 3.10781, training accuracy: 0.84800: 100%|██████████| 125/125 [01:32<00:00,  1.35batch/s]


[INFO] val_loss has been improved from 0.47659 to 0.41460. Saving Model!
epoch 02, loss: 0.38848, acc: 0.84800, val_loss: 0.41460, val_accuracy: 0.83100


training loss: 1.89928, training accuracy: 0.90600: 100%|██████████| 125/125 [01:32<00:00,  1.35batch/s]


[INFO] val_loss has been improved from 0.41460 to 0.40711. Saving Model!
epoch 03, loss: 0.23741, acc: 0.90600, val_loss: 0.40711, val_accuracy: 0.85200


training loss: 1.12802, training accuracy: 0.94900: 100%|██████████| 125/125 [01:32<00:00,  1.35batch/s]


epoch 04, loss: 0.14100, acc: 0.94900, val_loss: 0.49461, val_accuracy: 0.85000


training loss: 0.64777, training accuracy: 0.96800: 100%|██████████| 125/125 [01:32<00:00,  1.35batch/s]


epoch 05, loss: 0.08097, acc: 0.96800, val_loss: 0.51634, val_accuracy: 0.84400


In [29]:
# Load the saved state_dict.
# 저장한 state_dict를 로드 합니다.
BERT_model.load_state_dict(torch.load(f'{model_name}.pth'))

<All keys matched successfully>

### Inference

In [30]:
class Predictor():
    def __init__(self, model, tokenizer, labels: dict):
        self.model = model
        self.tokenizer = tokenizer
        self.labels = labels

    def predict(self, sentence):
        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                # 1개 문장
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )
        tokens.to(device)
        prediction = self.model(**tokens) # (1, 2)

        prediction = F.softmax(prediction, dim=1) # (1,2)

        output = prediction.argmax(dim=1).item()

        prob, result = prediction.max(dim=1)[0].item(), self.labels[output]
        print(f'[{result}]\n확률은: {prob*100:.3f}% 입니다.')

In [31]:
# Create a Huggingface tokenizer
# Huggingface 토크나이저 생성
tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)

labels = {
    0: '부정 리뷰 입니다.',
    1: '긍정 리뷰 입니다.'
}

# Create a Predictor instance.
# Predictor 인스턴스를 생성합니다.
predictor = Predictor(BERT_model, tokenizer, labels)

In [32]:
# Create a simple function that can make a prediction on user input and produce an output.
# 사용자 입력에 대하여 예측 후 출력을 낼 수 있는 간단한 함수를 생성합니다.
def predict_sentence(predictor):
    input_sentence = input('문장을 입력해 주세요: ')
    predictor.predict(input_sentence)


In [33]:
# Example of entering a negative review
# 부정 리뷰 입력 예시
predict_sentence(predictor)

문장을 입력해 주세요: 이런 너무 아쉽지 않네요
[긍정 리뷰 입니다.]
확률은: 91.449% 입니다.


### 실습
1. fill in the blanks. (빈칸을 채워주세요.)
2. Save and post the result of 5 random sentences as a screenshot. (임의의 5문장의 결과를 스크린샷으로 저장해서 올려주세요.)


In [34]:
import pandas as pd

train = pd.read_csv('~/Korpora/nsmc/ratings_train.txt', sep='\t')
test = pd.read_csv('~/Korpora/nsmc/ratings_test.txt', sep='\t')
print('train',train.shape)
print('test',test.shape)
train['length'] = train['document'].apply(lambda x: len(str(x)))
test['length'] = test['document'].apply(lambda x: len(str(x)))


train = train.loc[train['length'] > 5]
train = train.sample(1000)
test = test.loc[test['length'] > 5]
test = test.sample(1000)

train (150000, 3)
test (50000, 3)


In [35]:
import torch
from transformers import BertTokenizerFast, AlbertModel
from torch.utils.data import Dataset, DataLoader


class NaverReviewDataset(Dataset):

    def __init__(self, dataframe, tokenizer_pretrained):
        # sentence, label 컬럼으로 구성된 데이터프레임 전달
        self.data = dataframe
        # Huggingface 토크나이저 생성

        self.tokenizer = BertTokenizerFast.from_pretrained(tokenizer_pretrained)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx): # 클래스의 인덱스에 접근할 때 자동으로 호출되는 메서드
        # iloc : 행번호로 선택하는 방법
        sentence = self.data.iloc[idx]['document']
        label = self.data.iloc[idx]['label']

        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                # 1개 문장
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )

        input_ids = tokens['input_ids'].squeeze(0)       # 2D -> 1D : 1,512 -> 512
        attention_mask = tokens['attention_mask'].squeeze(0) # 2D -> 1D
        token_type_ids = torch.zeros_like(attention_mask)

        # input_ids, attention_mask, token_type_ids 이렇게 3가지 요소를 반환하도록 합니다.
        # input_ids: 토큰
        # attention_mask: 실제 단어가 존재하면 1, 패딩이면 0 (패딩은 0이 아닐 수 있습니다)
        # token_type_ids: 문장을 구분하는 id. 단일 문장인 경우에는 전부 0
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'token_type_ids': token_type_ids,
        }, torch.tensor(label)

In [36]:
MODEL_NAME='kykim/albert-kor-base'
from transformers import AlbertModel

# Specify a tokenizer
# 토크나이저 지정
tokenizer_pretrained = MODEL_NAME

# Create train, test datasets
# train, test 데이터셋 생성
train_data = NaverReviewDataset(train, tokenizer_pretrained)
test_data = NaverReviewDataset(test, tokenizer_pretrained)

# Specify the Dataset you created earlier with DataLoader, setting the batch configuration, shuffle, num_workers, etc.
# DataLoader로 이전에 생성한 Dataset를 지정하여, batch 구성, shuffle, num_workers 등을 설정합니다.
train_loader = DataLoader(train_data, batch_size=8, shuffle=True, num_workers=2)
test_loader = DataLoader(test_data, batch_size=8, shuffle=False, num_workers=2)

# Train_loader and test_loader are iterators.
# An iterator is an object that uses repetition to perform some processing.
# train_loader과 test_loader는 iterator이다.
# iterator란 반복을 이용해 어떤 처리를 수행하는 객체를 말한다.
print('train_loader',train_loader)
print('test_loader',test_loader)

tokenizer_config.json:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/344k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result i

train_loader <torch.utils.data.dataloader.DataLoader object at 0x7cfb5d155690>
test_loader <torch.utils.data.dataloader.DataLoader object at 0x7cfb256a9360>


In [37]:
class FineTuningBertModel(nn.Module):
    def __init__(self, bert_pretrained, dropout_rate=0.5):
        super().__init__()
        # 1. specify pretrained model(사전학습 모델 지정)
        ###################################################
        self.bert = BertModel.from_pretrained(bert_pretrained)
        ###################################################
        # 2. set dropout(dropout 설정)
        ###################################################
        self.dropout = nn.Dropout(p=dropout_rate)
        ###################################################
        # 3. define the final output layer(최종 출력층 정의)
        ###################################################
        self.fc = nn.Linear(768, 2)
        ###################################################

    def forward(self, input_ids, attention_mask, token_type_ids):
        # Substitute the input into the pre-trained Bert model
        # 입력을 pre-trained bert model 로 대입
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        # 4. Get last_hidden_state of result(결과의 last_hidden_state 가져옴)
        ##############################################
        last_hidden_state = output['last_hidden_state']
        ##############################################
        # 5. Get [CLS] token([CLS] 토큰을 가져옴)
        ##############################################
        x = self.dropout(last_hidden_state[:, 0, :])
        ##############################################
        # Final output via FC
        # FC 을 거쳐 최종 출력
        x = self.fc(x)
        return x

albert_model = FineTuningBertModel(MODEL_NAME)
albert_model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(albert_model.parameters(), lr=1e-5)

from tqdm import tqdm  # Progress Bar 출력

def model_train(model, data_loader, loss_fn, optimizer, device):
    # Set the model to training mode. Gradient will be updated when in training mode. Be sure to change mode with train().
    # 모델을 훈련모드로 설정합니다. training mode 일 때 Gradient 가 업데이트 됩니다. 반드시 train()으로 모드 변경을 해야 합니다.
    model.train()

    # This is a temporary variable for calculating loss and accuracy. Initialize it to 0.
    # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
    running_loss = 0
    corr = 0
    counts = 0

    # Wrap it in tqdm to monitor the training status while outputting a pretty progress bar.
    # 예쁘게 Progress Bar를 출력하면서 훈련 상태를 모니터링 하기 위하여 tqdm으로 래핑합니다.
    prograss_bar = tqdm(data_loader, unit='batch', total=len(data_loader), mininterval=1)

    # Start learning mini-batch.
    # mini-batch 학습을 시작합니다.
    for idx, (inputs, labels) in enumerate(prograss_bar):
        # put inputs, label data on device (cuda:0 or cpu)
        # inputs, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
        inputs = {k:v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)


        # 6. Initialize the stacked Gradient.(누적 Gradient를 초기화 합니다.)
        ##################################################
        optimizer.zero_grad()
        ##################################################

        # Proceed with Forward Propagation to get the result.
        # Forward Propagation을 진행하여 결과를 얻습니다.
        output = model(**inputs)

        # Calculate the loss by substituting the values of output, label into the loss function.
        # 손실함수에 output, label 값을 대입하여 손실을 계산합니다.
        loss = loss_fn(output, labels)

        # Proceed with back propagation to compute the derivative.
        # 오차역전파(Back Propagation)을 진행하여 미분 값을 계산합니다.
        loss.backward()

        # update calculated gradient
        # 계산된 Gradient를 업데이트 합니다.
        optimizer.step()


        # 7. output.max(dim=1) returns max probability and max index(output의 max(dim=1)은 max probability와 max index를 반환합니다.)
        # Ignore max probability, and store max index in index to compare against the label value to derive the accuracy.
        # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
        ################################################################
        _, index = output.max(dim=1)
        ################################################################
        # index.eq(lbl).sum() computes the sum of the labels that match exactly. item() extracts a value from a tensor.
        # accumulate the sum in the corr variable.
        # index.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
        # 합계는 corr 변수에 누적합니다.
        corr += index.eq(labels).sum().item()
        counts += len(labels)


        # loss value is the average loss for one batch. labels.size(0) is the batch size.
        # multiply loss by labels.size(0) to calculate the total loss for one batch.
        # accumulate them and divide by the total number of datasets at the end of the epoch to get the mean loss.
        # loss 값은 1개 배치의 평균 손실(loss) 입니다. labels.size(0)은 배치사이즈(batch size) 입니다.
        # loss 와 labels.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
        # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
        running_loss += loss.item() * labels.size(0)

        # Update learning progress in the progress bar
        # 프로그레스바에 학습 상황 업데이트
        prograss_bar.set_description(f"training loss: {running_loss/(idx+1):.5f}, training accuracy: {corr / counts:.5f}")

    # The accumulated number of correct answers divided by the total number yields the accuracy.
    # 누적된 정답수를 전체 개수로 나누어 주면 정확도가 산출됩니다.
    acc = corr / len(data_loader.dataset)

    # Returns the mean loss and accuracy.
    # 평균 손실(loss)과 정확도를 반환합니다.
    # train_loss, train_acc
    return running_loss / len(data_loader.dataset), acc

You are using a model of type albert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


pytorch_model.bin:   0%|          | 0.00/53.3M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at kykim/albert-kor-base and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.lay

In [38]:
def model_evaluate(model, data_loader, loss_fn, device):
    # model.eval() sets the model to eval mode.
    # model.eval()은 모델을 평가모드로 설정을 바꾸어 줍니다.
    # 8. This is a necessary step in the evaluation process to change the role of a layer such as dropout.(dropout과 같은 layer의 역할 변경을 위하여 evaluation 진행시 꼭 필요한 절차 입니다.)
    ################################################################
    model.eval()
    ################################################################

    # This is required to prevent the gradient from being updated.
    # Gradient가 업데이트 되는 것을 방지 하기 위하여 반드시 필요합니다.
    with torch.no_grad():
        # This is a temporary variable for calculating loss and accuracy. Initialize it to 0.
        # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
        corr = 0
        running_loss = 0

        # Proceed evaluation for each batch.
        # 배치별 evaluation을 진행합니다.
        for inputs, labels in data_loader:
            # put inputs, label data on device (cuda:0 or cpu)
            # inputs, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
            inputs = {k:v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)

            # Proceed with Forward Propagation to get the result.
            # 모델에 Forward Propagation을 하여 결과를 도출합니다.
            output = model(**inputs)

            # 9. output.max(dim=1) returns max probability and max index(output의 max(dim=1)은 max probability와 max index를 반환합니다.)
            # Ignore max probability, and store max index in index to compare against the label value to derive the accuracy.
            # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
            ################################################################
            _, index = output.max(dim=1)
            ################################################################

            # index.eq(lbl).sum() computes the sum of the labels that match exactly. item() extracts a value from a tensor.
            # accumulate the sum in the corr variable.
            # index.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
            # 합계는 corr 변수에 누적합니다.
            corr += torch.sum(index.eq(labels)).item()

            # loss value is the average loss for one batch. labels.size(0) is the batch size.
            # multiply loss by labels.size(0) to calculate the total loss for one batch.
            # accumulate them and divide by the total number of datasets at the end of the epoch to get the mean loss.
            # loss 값은 1개 배치의 평균 손실(loss) 입니다. img.size(0)은 배치사이즈(batch size) 입니다.
            # loss 와 img.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
            # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
            running_loss += loss_fn(output, labels).item() * labels.size(0)

        # Calculate the validation accuracy.
        # Divide the accumulated number of correct answers by the number in the entire dataset to get the final accuracy.
        # validation 정확도를 계산합니다.
        # 누적한 정답숫자를 전체 데이터셋의 숫자로 나누어 최종 accuracy를 산출합니다.
        acc = corr / len(data_loader.dataset)

        # return outputs
        # 결과를 반환합니다.
        # val_loss, val_acc
        return running_loss / len(data_loader.dataset), acc

In [55]:
# specify epochs
# 최대 Epoch을 지정합니다.
num_epochs = 5 #이거 5으로 다시 고

# 10. # Define the name of the model to save as a checkpoint.(checkpoint로 저장할 모델의 이름을 정의 합니다.)
###########################################
model_name = 'bert-kor-base'
###########################################

min_loss = np.inf

# Perform training and validation.
# Epoch 별 훈련 및 검증을 수행합니다.
for epoch in range(num_epochs):
    # Model Training
    # 훈련 손실과 정확도를 반환 받습니다.
    train_loss, train_acc = model_train(albert_model, train_loader, loss_fn, optimizer, device)

    # 검증 손실과 검증 정확도를 반환 받습니다.
    val_loss, val_acc = model_evaluate(albert_model, test_loader, loss_fn, device)

    # 11. If val_loss has improved, update min_loss and save the model's weights.(val_loss 가 개선되었다면 min_loss를 갱신하고 model의 가중치(weights)를 저장합니다.)
    ###########################################################################
    if val_loss < min_loss:
        print(f'[INFO] val_loss has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!')
        min_loss = val_loss
        torch.save(BERT_model.state_dict(), f'{model_name}.pth')
    ###########################################################################

    # print results for each epoch
    # Epoch 별 결과를 출력합니다.
    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, acc: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}')

  0%|          | 0/125 [00:00<?, ?batch/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
training loss: 3.52396, training accuracy: 0.81100:  98%|█████████▊| 123/125 [00:51<00:00,  2.48batch/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
training loss: 3.52396, training accuracy: 0.81100: 100%|██████████| 125/125 [00:51<00:00,  2.44batch/s]


[INFO] val_loss has been improved from inf to 0.38412. Saving Model!
epoch 01, loss: 0.44050, acc: 0.81100, val_loss: 0.38412, val_accuracy: 0.83900


training loss: 0.90898, training accuracy: 0.95300: 100%|██████████| 125/125 [00:50<00:00,  2.46batch/s]


epoch 02, loss: 0.11362, acc: 0.95300, val_loss: 0.48669, val_accuracy: 0.82800


training loss: 0.30777, training accuracy: 0.99000: 100%|██████████| 125/125 [00:50<00:00,  2.48batch/s]


epoch 03, loss: 0.03847, acc: 0.99000, val_loss: 0.50204, val_accuracy: 0.83500


training loss: 0.15148, training accuracy: 0.99500: 100%|██████████| 125/125 [00:50<00:00,  2.48batch/s]


epoch 04, loss: 0.01894, acc: 0.99500, val_loss: 0.63750, val_accuracy: 0.84000


training loss: 0.06634, training accuracy: 0.99800: 100%|██████████| 125/125 [00:50<00:00,  2.47batch/s]


epoch 05, loss: 0.00829, acc: 0.99800, val_loss: 0.72246, val_accuracy: 0.84100


In [56]:
# Load the saved state_dict.
# 저장한 state_dict를 로드 합니다.
import torch

original_state_dict=torch.load(f'{model_name}.pth')
original_position_embeddings=original_state_dict['bert.embeddings.position_embeddings.weight']
resized_position_embeddings=original_position_embeddings[:256,:]
original_state_dict['bert.embeddings.position_embeddings.weight']=resized_position_embeddings

albert_model.load_state_dict(original_state_dict)

<All keys matched successfully>

In [57]:
class Predictor():
    def __init__(self, model, tokenizer, labels: dict):
        self.model = model
        self.tokenizer = tokenizer
        self.labels = labels

    def predict(self, sentence):
        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                # 1개 문장
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )
        tokens.to(device)
        prediction = self.model(**tokens) # (1, 2)

        prediction = F.softmax(prediction, dim=1) # (1,2)

        output = prediction.argmax(dim=1).item()

        prob, result = prediction.max(dim=1)[0].item(), self.labels[output]
        print(f'[{result}]\n확률은: {prob*100:.3f}% 입니다.')

In [58]:
# Create a Huggingface tokenizer
# Huggingface 토크나이저 생성
tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)

labels = {
    0: '부정 리뷰 입니다.',
    1: '긍정 리뷰 입니다.'
}
# Create a Predictor instance
# Predictor 인스턴스를 생성합니다.
predictor = Predictor(albert_model, tokenizer, labels)

# Create a simple function that can make a prediction on user input and produce an output.
# 사용자 입력에 대하여 예측 후 출력을 낼 수 있는 간단한 함수를 생성합니다.
def predict_sentence(predictor):
    input_sentence = input('문장을 입력해 주세요: ')
    predictor.predict(input_sentence)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


In [59]:
#12. Check results : Run for 5 sentences(결과 확인 : 5문장에 대해서 실행하세요)
###########################
predict_sentence(predictor)
predict_sentence(predictor)
predict_sentence(predictor)
predict_sentence(predictor)
predict_sentence(predictor)
###########################

문장을 입력해 주세요: 너무 재밌네요
[긍정 리뷰 입니다.]
확률은: 99.586% 입니다.
문장을 입력해 주세요: 너무 재미없지 않네요
[긍정 리뷰 입니다.]
확률은: 87.119% 입니다.
문장을 입력해 주세요: 한번 더 볼거 같지 않아요
[부정 리뷰 입니다.]
확률은: 94.776% 입니다.
문장을 입력해 주세요: 러닝 타임동안 재밌게 본거 같아요
[긍정 리뷰 입니다.]
확률은: 99.402% 입니다.
문장을 입력해 주세요: 정말 이해할 수 없는 부분이 있었지만 반전이 좋네요
[긍정 리뷰 입니다.]
확률은: 99.384% 입니다.
